In [77]:
import os
import sys
import pandas as pd
from pyspark import SparkContext, sql, SparkConf
import nltk
import re
from datetime import datetime
from itertools import islice
import __builtin__
import numpy as np
from ast import literal_eval

#import pyspark.sql as sparksql
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Row

#-------------------------------
# Importing from other folders, appending the path
sys.path.append('../')
from spark_sessions import start_session, stop_session

In [188]:
spark_configs = (SparkConf()\
            .setAppName("experiment-airbnb")\
            .set("spark.executor.instances", "10")\
            .set("spark.executor.cores", 2)\
            .set("spark.dynamicAllocation.enabled", "false")\
            .set("spark.shuffle.service.enabled", "false")\
            .set("spark.executor.memory", "500MB"))

#----------------
def panda_framing(self):
    # Frame the data with panda
    _panda_framed = pd.read_csv(sample)
    panda_framed = pd.DataFrame(_panda_framed)
    
    return panda_framed

#----------------
def spark_session(version):
    if version == 0:
        conf = spark_configs
        return conf
    
    elif version == 1:
        spark = SparkSession.builder \
            .master("yarn") \
            .appName("experiment-airbnb") \
            .enableHiveSupport() \
            .getOrCreate()
            
        return spark
            

#----------------
def spark_data_txt(sample):
    # Creating "pyspark.context.SparkContext object"
    conf = spark_session(version=0)
    sc = SparkContext.getOrCreate(conf = conf)
    data = sc.textFile(sample)
    #print 'type of context for session v 0:', sc
    return data
    
#----------------
def spark_data_sql(sample):
    # Creating "pyspark.sql.context.SQLContext object"
    sc = SQLContext(spark_session(version=1))
    data = sc.read.load(sample,
                    format='com.databricks.spark.csv', 
                    header='true', 
                    inferSchema='true').cache()
    #print 'type of context for session v 1:', sc
        
    return data

#----------------
def structure_field(key, item): # item = StringType() ...
    key = str
    item = str
    structure = []
    structure.append(StructField(key, item, True))
        
    return structure

#----------------
def spark_schema_v1(sample):
    data = spark_data_txt(sample)
    _keys = data.take(2)
#     keys_list = []
#     for _key in _keys[1:]:
#         print _key.split(',')
#     keys_list.append(_key)
    
#         #print "keys:", nltk.word_tokenize(_key)#_key.replace('"','')#.replace(',','')
        #print  u', '.join(_keys[1:])
            
    #print "token", nltk.word_tokenize(_keys[1:])#.replace('"','')
    schemaString = data.first().replace('"','') # Creating a schema
    #print schemaString
    #for i in schemaString.split(','): print i
    fields = [StructField(field_name, StringType(), True) 
              for field_name in schemaString.split(',')
             ]   
    _schema = [str(key) for key in schemaString.split(',')] # mapping
    #print fields

#----------------
def residual_data():
    data = spark_data_txt(sample)
    print data.columns()
    headers = data.filter(lambda l: "_id" in l)
    _data = data.subtract(headers)
#     _temp_data = _data.\
#         map(lambda k: k.split(",")).\
#         map(lambda p: (
#             p[0], p[1], 
#             parse(p[2].strip('"')), 
#             float(p[3]), 
#             float(p[4]) , 
#             p[5], 
#             p[6] , 
#             int(p[7]), 
#             parse(p[8].strip('"')), 
#             float(p[9]), 
#             float(p[10]), 
#             int(p[11]), 
#             p[12], 
#             float(p[13]), 
#             int(p[14]), 
#             p[15] ))
# >>> taxi_temp.top(2)  # have a look:
#     print _data.take(2)
#     _spark_data = _data.subtract(headers)
#     spark_data= _spark_data.map(lambda l: l.split(","))

    
#----------------
def spark_schema_v0(sample):
    data = spark_data_sql(sample) # we need this to create the keys (data types)
    _keys = data.take(2)
    
    _data = spark_data_txt(sample) # we need this to create the schema
    schemaString = _data.first().replace('"','')
    _schema = schemaString.split(',')#data.columns # Creating a schema
    headers = _data.filter(lambda l: "_id" in l) # Headers needs to be an RDD - the string we constructed above will not do the job
    headers.collect()
    _spark_data = _data.subtract(headers)
    spark_data= _spark_data.map(lambda l: l.split(","))

    
    print "Found %d columns in your data file \n" % len(_keys[0]) 
    _types = []
    fields = []
    #schem = schemaString.split(',')
    print "Followings are the details of your schema:"
    print "------------------------------------------"
    for key in range(len(_keys[1])) or (type(_keys[1][key]) == None):   
        if (type(_keys[1][key]) == int):
            data_type = IntegerType()
            str_struct = StructField(_schema[key], data_type, True)
            fields.append(str_struct)
            #_types.append((_schema[key], data_type))
            print 'key:', '"'+_schema[key]+'",',',', ' value:', _keys[1][key],',', ' data type:', data_type #, ' value:', _keys[0][key], 
        
        elif (type(_keys[1][key]) == float):
            data_type = FloatType()
            str_struct = StructField(_schema[key], data_type, True)
            fields.append(str_struct)
            #_types.append((_schema[key], data_type))
            print 'key:', '"'+_schema[key]+'",',',',' value:', _keys[1][key],',', ' data type:', data_type #, ' value:', _keys[0][key],
        
        elif (type(_keys[1][key]) == unicode) or (type(_keys[0][key]) == str):
            data_type = StringType()
            str_struct = StructField(_schema[key], data_type, True)
            fields.append(str_struct)
            #_types.append((_schema[key], data_type))
            print 'key:', '"'+_schema[key]+'",',',', ' value:', _keys[1][key], ',', 'data type:', data_type
        
        else:# (type(_keys[0][key]) == unicode) or (type(_keys[0][key]) == str):
            data_type = TimestampType()
            str_struct = StructField(_schema[key], data_type, True)
            fields.append(str_struct)
            #_types.append((_schema[key], data_type))
            print 'key:', '"'+_schema[key]+'",',',', ' value:', _keys[1][key], ',', 'data type:', data_type
    
    print "------------------------------------------"
    print "Your schema's 'StructField' are:"
    for x in fields:
        print x
    
    print "------------------------------------------"

    
    # Creating the spark DataFrame:
    conf = spark_session(version=0)
    sc = SparkContext.getOrCreate(conf = conf)
    _spark_data.map(lambda l: tuple(l[:1]) + tuple(l[1].split(','))).map(lambda x: x.split('\t'))
    print _spark_data
    #rdd = sc.parallelize(_spark_data.map( lambda elem: list(elem)))
    #.map(lambda p: (p[0], p[1].strip()))
    
    schema = StructType(fields)
    spark_df = _spark_data.map(lambda x: (x, )).toDF(schema)
#     spark_df.head(5)
    
#     try:
#         conf = spark_session(version=0)
#         sc = SparkContext.getOrCreate(conf = conf)
#         rdd = sc.parallelize(spark_data)
    #spark_df = spark_session(version=1).createDataFrame(spark_data,schema)
#     spark_df.show()
#     except Exception as e:
#         print (e)
    

sample = "/Users/Ahmed/Documents/DataMining_Stuff/Hadoop/Spark/PySpark/data/airbnb/sample/sample.csv"
# spark_data_txt(sample)
# spark_data_sql(sample)
residual_data()

#spark_schema_v0(sample)
#spark_schema_v1(sample)

AttributeError: 'RDD' object has no attribute 'columns'